# Scratch Model

## Importings

In [ ]:
# Standard library imports
from functools import partial
from pathlib import Path
from typing import Any, Self
import datetime
import os
import json

import matplotlib.pyplot as plt
import pandas as pd

# Keras imports (Core)
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Input, Lambda, Flatten, Dropout

from keras.layers import RandomTranslation, RandomRotation, RandomZoom, RandomFlip, RandomContrast

from keras.optimizers import SGD, Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.losses import CategoricalCrossentropy
from keras.metrics import CategoricalAccuracy, AUC, F1Score 
from keras.callbacks import ModelCheckpoint, CSVLogger, LearningRateScheduler
from keras.utils import image_dataset_from_directory

# Keras Applications imports
from keras.applications.xception import Xception, preprocess_input



## Collab Setting
If you are using Google Colab, run the following cell to set up google drive mount. 

Also remember to change paths when needed. 


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

## If using Google Colab, set the data_dir_path to the mounted drive
# root_dir_path = Path('/content/drive/MyDrive')

root_dir_path = Path(".")

## Auxiliary Functions


In [ ]:
def load_ds(
    dir_path: Path,
    batch_size: int,
    input_shape: tuple[int, ...],
    shuffle: bool = True
) -> Any:

    height, width, n_channels = input_shape
    image_size = (height, width)

    ds = image_dataset_from_directory(
        dir_path,
        label_mode="categorical",
        batch_size=batch_size,
        image_size=image_size,
        interpolation="bilinear",
        shuffle=shuffle,
        verbose=False
    )

    return ds

In [ ]:
def exp_decay_lr_scheduler(
    epoch: int,
    current_lr: float,
    factor: float = 0.95
) -> float:

    current_lr *= factor

    return current_lr

## Preprocess the Data

In [ ]:
...

## Define the model

In [ ]:
def create_model(input_shape=(299, 299, 3), n_classes=202, model_name="UnnamedModel") -> Self:
   
    # Define the input tensor
    inputs = Input(shape=input_shape)

    # Augmentation layer

    x = RandomRotation(0.2)(x)
    x = RandomTranslation(0.1, 0.1)(x)
    x = RandomZoom(0.1)(x)
    x = RandomFlip("horizontal")(x)
    x = RandomContrast(0.2)(x)

    
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.3)(x)
    outputs = Dense(n_classes, activation='softmax')(x)

    # Create the complete model
    model = Model(inputs=inputs, outputs=outputs, name="UnnamedModel")

    return model

## Define Train Parameters

In [ ]:
# Base Model Independent Parameters
n_classes = 202
batch_size = 64
epochs = 10

input_shape = (299, 299, 3)
image_size = (299, 299)
value_range = (-1.0, 1.0)

## Learning Rate

In [8]:
initial_lr = 0.001
final_lr = 0.0003
factor = (final_lr / initial_lr) ** (1 / epochs)
lr_scheduler = partial(exp_decay_lr_scheduler, factor=factor)

## Evaluation Metrics

In [ ]:
categorical_accuracy = CategoricalAccuracy(name="accuracy")
precision = CategoricalAccuracy(name="precision")
recall = CategoricalAccuracy(name="recall")
f1_score = F1Score(average="macro", name="f1_score")
metrics = [categorical_accuracy,precision, recall, f1_score]

## Load Data

In [ ]:
data_dir_path = root_dir_path / "data"

train_dir_path = data_dir_path / "train"
val_dir_path = data_dir_path / "val"
test_dir_path = data_dir_path / "test"

_load_ds = partial(
        load_ds, batch_size=batch_size, input_shape=input_shape
    )

train_ds = _load_ds(train_dir_path)
val_ds = _load_ds(val_dir_path, shuffle=False)
test_ds = _load_ds(test_dir_path, shuffle=False)

## Training

In [ ]:
model = create_model()
optimizer = Adam(learning_rate=initial_lr)
loss = CategoricalCrossentropy() 

model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
#model.summary()

In [ ]:
timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Define callbacks
metrics_file_path = root_dir_path / f"{timestamp}_{model.name}_metrics.csv"
checkpoint_file_path = root_dir_path / f"{timestamp}_{model.name}_checkpoint.keras"

checkpoint_callback = ModelCheckpoint(
    checkpoint_file_path, monitor="val_loss", verbose=0
)
metrics_callback = CSVLogger(metrics_file_path)
lr_scheduler_callback = LearningRateScheduler(lr_scheduler)
callbacks = [checkpoint_callback, metrics_callback, lr_scheduler_callback]


# Fit the model
_ = model.fit(
    train_ds, validation_data=val_ds, epochs=epochs, callbacks=callbacks, verbose=1
)

evaluation_dict = model.evaluate(test_ds, return_dict=True, verbose=0)

print(evaluation_dict)

## Ploting the metrics

In [ ]:
df_metrics = pd.read_csv(metrics_file_path)

df_metrics.head()

# Load the metrics CSV file
metrics_df = pd.read_csv(metrics_file_path)

# Create a figure with 2 rows and 2 columns
fig, axs = plt.subplots(2, 3, figsize=(15, 10))

# Plot Loss
axs[0, 0].plot(metrics_df['epoch'], metrics_df['loss'], 'b-', label='Training')
axs[0, 0].plot(metrics_df['epoch'], metrics_df['val_loss'], 'r-', label='Validation')
axs[0, 0].set_title('Loss')
axs[0, 0].set_xlabel('Epoch')
axs[0, 0].set_ylabel('Loss')
axs[0, 0].legend()

# Plot Accuracy
axs[0, 1].plot(metrics_df['epoch'], metrics_df['accuracy'], 'b-', label='Training')
axs[0, 1].plot(metrics_df['epoch'], metrics_df['val_accuracy'], 'r-', label='Validation')
axs[0, 1].set_title('Accuracy')
axs[0, 1].set_xlabel('Epoch')
axs[0, 1].set_ylabel('Accuracy')
axs[0, 1].legend()


# Plot F1 Score
axs[1, 0].plot(metrics_df['epoch'], metrics_df['f1_score'], 'b-', label='Training')
axs[1, 0].plot(metrics_df['epoch'], metrics_df['val_f1_score'], 'r-', label='Validation')
axs[1, 0].set_title('F1 Score')
axs[1, 0].set_xlabel('Epoch')
axs[1, 0].set_ylabel('F1 Score')
axs[1, 0].legend()

# Plot Precision
axs[1, 1].plot(metrics_df['epoch'], metrics_df['precision'], 'b-', label='Training')
axs[1, 1].plot(metrics_df['epoch'], metrics_df['val_precision'], 'r-', label='Validation')
axs[1, 1].set_title('Precision')
axs[1, 1].set_xlabel('Epoch')
axs[1, 1].set_ylabel('Precision')
axs[1, 1].legend()

# Plot Precision
axs[2, 0].plot(metrics_df['epoch'], metrics_df['recall'], 'b-', label='Training')
axs[2, 0].plot(metrics_df['epoch'], metrics_df['val_recall'], 'r-', label='Validation')
axs[2, 0].set_title('Recall')
axs[2, 0].set_xlabel('Epoch')
axs[2, 0].set_ylabel('Recall')
axs[2, 0].legend()

plt.tight_layout()
plt.savefigroot_dir_path / "models" / f"{timestamp}_{model.name}_training_metrics.png", dpi=300
plt.show()


## Saving Model (WIP)

In [ ]:
# Create a timestamped model name
model_save_path = root_dir_path / 'models' / f"{timestamp}_{model.name}.keras"

# Save the entire model
model.save(model_save_path)
print(f"Model saved to {model_save_path}")

Model saved to models\20250401_220339_AugmentedXception.keras


In [32]:
for n in metrics:
    print(n.name)

accuracy
auc
f1_score


In [ ]:
# Bundle your config into a dict
config = {
    "n_classes": n_classes,
    "batch_size": batch_size,
    "epochs": epochs,
    "input_shape": input_shape,
    "image_size": image_size,
    "value_range": value_range,
    "initial_lr": initial_lr,
    "final_lr": final_lr,
    #"lr_scheduler": lr_scheduler,
    "metrics": [metric.name for metric in metrics],
    "optimizer": optimizer.name,
    "loss": loss.name,
}

config_save_path = root_dir_path / 'models' / f"{timestamp}_{model.name}_config.json"
# Save to a pickle file
with open(config_save_path, "w") as f:
    json.dump(config, f)

In [ ]:
with open(config_save_path, "r") as f:
    loaded_config = json.load(f)


loaded_config

{'n_classes': 202,
 'batch_size': 64,
 'epochs': 10,
 'input_shape': [299, 299, 3],
 'image_size': [299, 299],
 'value_range': [-1.0, 1.0],
 'initial_lr': 0.001,
 'final_lr': 0.0003,
 'factor': 0.8865681505652133,
 'metrics': ['accuracy', 'auc', 'f1_score'],
 'optimizer': 'adam',
 'loss': 'categorical_crossentropy'}